In [1]:
import os
import re
import csv
import h5py
import warnings
import tqdm
import numpy as np
import pandas as pd

In [201]:
class VSS_File:
    # Class for vibration-based soft sensing database in an hdf5 file
    def __init__(self, filePath):
        self.path = filePath
        self._index = 0
        self._fileh5ref = h5py.File(filePath,'r')
        self.units =  [self.VSS_Unit_Reference(self,self._fileh5ref[group]) for group in self._fileh5ref.keys()]

    def __repr__(self):
        return f"Vibration-based database for ({len(self.units)} units)"
           
    def __iter__(self):
        return self
    
    def __next__(self):
        if self._index == len(self.units):
            self._index = 0
            raise StopIteration
        else:
            self._index += 1
            return self.units[self._index-1]

    class VSS_Unit_Reference:
        # Class for a unit group inside a hdf5 file
        def __init__(self, parent, unitGroupId:h5py.Group):
            self._h5ref = unitGroupId
            self._h5file = parent
            self.name = unitGroupId.name
            self._index = 0
            self.tests = [self.VSS_Test_Reference(self,self._h5ref[group]) for group in self._h5ref.keys()]

        def __repr__(self):
            return f"Vibration-based database for unit <{self.name}> ({len(self.tests)} tests)"
    
        def __str__(self):
            return self.name
        
        def __iter__(self):
            return self
        
        def __next__(self):
            if self._index == len(self.tests):
                self._index = 0
                raise StopIteration
            else:
                self._index += 1
                return self.tests[self._index-1]
            
        def filterTestsByAttribute(self, attribute, value):
            match attribute:
                case "angularSpeed":
                    # 2100 2475 2850 3225 3600 RPM
                    return [test for test in self.tests if test._h5ref.attrs['angularSpeed'] in value]
                case "condensingTemperature" | "evaporatingTemperature":
                    # condensingTemperature 34º C até 54 ºC
                    # evaporatingTemperature 10º C até 30 ºC
                    return [test for test in self.tests if min(value) <= test._h5ref.attrs[attribute] <= max(value)]
                # case "repetition":
                #     #  n
                #     pass
                case "type":
                    # A = mapa principal
                    # B = mapa secundário
                    return [test for test in self.tests if test._h5ref.attrs['type'] == value]
                
        def filterTestsByAttributeDict(self, attributeDict):
                output = [test for test in self.tests]
                if "angularSpeed" in attributeDict.keys():
                    output = [test for test in output if int(test._h5ref.attrs['angularSpeed']) \
                               in attributeDict['angularSpeed']]
                else:
                    pass

                if "condensingTemperature" in attributeDict.keys():
                    output = [test for test in output if \
                             min(attributeDict['condensingTemperature']) \
                                <= float(test._h5ref.attrs['condensingTemperature'].replace(',','.')) \
                                    <= max(attributeDict['condensingTemperature'])]
                else:
                    pass

                if "evaporatingTemperature" in attributeDict.keys():
                    output = [test for test in output if \
                             min(attributeDict['evaporatingTemperature']) \
                                <= float(test._h5ref.attrs['evaporatingTemperature'].replace(',','.')) \
                                    <= max(attributeDict['evaporatingTemperature'])]
                else:
                    pass

                # if "repetition" in attributeDict.keys():
                #     pass
                # else:
                #     pass

                # if "type" in attributeDict.keys():
                #     pass
                # else:
                #     pass
                return output


        class VSS_Test_Reference:
            # Class for a test group inside a hdf5 file
            def __init__(self, parent, testGroupId:h5py.Group):
                self._h5ref = testGroupId
                self._h5file = parent._h5file
                self.h5unit = parent
                self.date = testGroupId.name
                self.unit = parent.name
                self.name = testGroupId.name

            def __repr__(self):
                return f"Vibration soft sensing test database <{self.name}>"
    
            def __str__(self):
                return (self.name)
            


datasetPath = "D:/Rafael/Dados/dataset.hdf5"

dataset = VSS_File(datasetPath)
        

In [213]:
filterAttributes = {
    "angularSpeed": [2100],
    "condensingTemperature": [40, 50]
}

len(dataset.units[0].filterTestsByAttributeDict(filterAttributes))

dataset.returnDataframe(filterAttributes)


328

In [71]:
vec = []
for unit in dataset.units:
    vec.append(len(unit.tests))
    # for test in unit.tests:
    #     print(test._h5file) 2100 2475 2850 3225 3600

sum(vec)
# len(dataset.units[0].filterTestsByAttribute("type", "B"))



# attributes = ["angularSpeed", "condensingTemperature"]
# values = ["2100", [40, 50]]

# for attribute, value in filterAttributes.items():
#     match attribute:
#         case "angularSpeed":
#             # 2100 2475 2850 3225 3600 RPM
#             return [test for test in self.tests if test._h5ref.attrs['angularSpeed'] in value]
#         case "condensingTemperature" | "evaporatingTemperature":
#             # condensingTemperature 34º C até 54 ºC
#             # evaporatingTemperature 10º C até 30 ºC
#             return [test for test in self.tests if min(value) <= test._h5ref.attrs[attribute] <= max(value)]
#         # case "repetition":
#         #     #  n
#         #     pass
#         case "type":
#             # A = mapa principal
#             # B = mapa secundário
#             return [test for test in self.tests if test._h5ref.attrs['type'] == value]

NameError: name 'tests' is not defined

In [21]:
import numpy as np


# pra acessar os datasets
# unit.tests[0]._h5ref.attrs['angularSpeed'] == '2100'


# for test in dataset.units[0]:
#     print(test)


# filtered =  [test.name for test in dataset.units[0] if test._h5ref.attrs['angularSpeed'] in '2100']


for testType in ['A','B']:
    print(testType)


A
B


In [ ]:
list(test._h5ref.attrs)

['angularSpeed',
 'condensingTemperature',
 'evaporatingTemperature',
 'repetition',
 'type']